# Solution-1
This tutorial shows how to identify drug molecules in the PDB by joining two datasets: 

1. Drug information from DrugBank
2. Ligand information from RCSB PDB

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring_index
from mmtfPyspark.datasets import pdbjMineDataset
from mmtfPyspark.webfilters import PdbjMineSearch
from mmtfPyspark.interactions import InteractionFilter, InteractionFingerprinter
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.structureViewer import view_binding_site
from ipywidgets import interact
from IPython.display import Markdown, display
import py3Dmol

#### Configure Spark

In [2]:
spark = SparkSession.builder.master("local[4]").appName("2-JoiningDatasets").getOrCreate()
sc = spark.sparkContext

##
[See examples](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/PDBMetaDataDemo.ipynb)
[SIFTS demo](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/SiftsDataDemo.ipynb)

For our first task, we need to run a taxonomy query. To figure out how to query for taxonomu, the command below lists the first 10 entries for the SIFTS taxonomy table. As you can see, we can use the science_name field to query for a specific organism.

In [3]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy LIMIT 10"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery)
taxonomy.show()

+-----+-----+------+--------------------+----------------+
|pdbid|chain|tax_id|     scientific_name|structureChainId|
+-----+-----+------+--------------------+----------------+
| 101M|    A|  9755|               PHYCD|          101M.A|
| 101M|    A|  9755|    Physeter catodon|          101M.A|
| 101M|    A|  9755|Physeter catodon ...|          101M.A|
| 101M|    A|  9755|Physeter catodon ...|          101M.A|
| 101M|    A|  9755|Physeter macrocep...|          101M.A|
| 101M|    A|  9755|         Sperm whale|          101M.A|
| 101M|    A|  9755|         sperm whale|          101M.A|
| 102L|    A| 10665|                BPT4|          102L.A|
| 102L|    A| 10665|    Bacteriophage T4|          102L.A|
| 102L|    A| 10665|Enterobacteria ph...|          102L.A|
+-----+-----+------+--------------------+----------------+



In [4]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy WHERE scientific_name = 'Mus musculus'"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery)
taxonomy.show(10)

+-----+-----+------+---------------+----------------+
|pdbid|chain|tax_id|scientific_name|structureChainId|
+-----+-----+------+---------------+----------------+
| 12E8|    H| 10090|   Mus musculus|          12E8.H|
| 12E8|    L| 10090|   Mus musculus|          12E8.L|
| 12E8|    M| 10090|   Mus musculus|          12E8.M|
| 12E8|    P| 10090|   Mus musculus|          12E8.P|
| 15C8|    H| 10090|   Mus musculus|          15C8.H|
| 15C8|    L| 10090|   Mus musculus|          15C8.L|
| 1914|    A| 10090|   Mus musculus|          1914.A|
| 1A0Q|    H| 10090|   Mus musculus|          1A0Q.H|
| 1A0Q|    L| 10090|   Mus musculus|          1A0Q.L|
| 1A14|    H| 10090|   Mus musculus|          1A14.H|
+-----+-----+------+---------------+----------------+
only showing top 10 rows



In [5]:
path = "../resources/mmtf_full_sample/"

pdb = mmtfReader.read_sequence_file(path, sc)

In [6]:
pdb = pdb.filter(PdbjMineSearch(taxonomyQuery))

In [7]:
interactionFilter = InteractionFilter(distanceCutoff=4.5, minInteractions=10)

interactions = InteractionFingerprinter.get_polymer_interactions(pdb, interactionFilter).cache()

In [8]:
interactions = interactions.withColumn("structureId", substring_index(interactions.structureChainId, '.', 1)).cache()
interactions.show()

+----------------+------------+-------------+--------------------+--------------------+--------------------+-----------+
|structureChainId|queryChainId|targetChainId|        groupNumbers|     sequenceIndices|            sequence|structureId|
+----------------+------------+-------------+--------------------+--------------------+--------------------+-----------+
|          5YQG.C|           F|            C|[132, 133, 177, 1...|[62, 137, 138, 18...|GPGSEFMGDREQLLQRA...|       5YQG|
|          5YQG.D|           F|            D|[125, 132, 133, 1...|[55, 62, 130, 137...|GPGSEFMGDREQLLQRA...|       5YQG|
|          1GCQ.A|           C|            A|[165, 167, 168, 1...|[8, 10, 11, 13, 1...|GSTYVQALFDFDPQEDG...|       1GCQ|
|          1GCQ.C|           A|            C|[606, 607, 608, 6...|[15, 16, 17, 18, ...|GSHMPKMEVFQEYYGIP...|       1GCQ|
|          1GCQ.B|           C|            B|[162, 163, 164, 1...|[5, 6, 7, 8, 22, ...|GSTYVQALFDFDPQEDG...|       1GCQ|
|          1GCQ.C|           B| 

## Visualize drug binding sites

#### Extract id columns as lists (required for visualization)

In [ ]:
structure_ids = interactions.select("structureId").rdd.flatMap(lambda x: x).collect()
query_chain_ids = interactions.select("queryChainID").rdd.flatMap(lambda x: x).collect()
target_chain_ids = interactions.select("targetChainID").rdd.flatMap(lambda x: x).collect()

Disable scrollbar for the visualization below

In [ ]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

#### Show binding site residues within 4.5 A from the drug molecule

In [ ]:
def view_protein_protein_interactions(structure_ids, query_chain_ids, target_chain_ids, distance=4.5):
    
    def view3d(i=0):
        
        print(f"PDB: {structure_ids[i]}, query: {query_chain_ids[i]}, target: {target_chain_ids[i]}")

        query = {'chain': query_chain_ids[i], 'chain': target_chain_ids[i], 'byres': 'true', 'expand': distance}
        target = {'chain': target_chain_ids[i], 'chain': query_chain_ids[i], 'byres': 'true', 'expand': distance}
        viewer = py3Dmol.view(query='pdb:' + structure_ids[i]) 
        viewer.setStyle(query, {'stick': {'colorscheme': 'orangeCarbon'}});
        viewer.setStyle(target, {'stick': {'colorscheme': 'lightblueCarbon'}})
        viewer.zoomTo(query)

        return viewer.show()

    s_widget = IntSlider(min=0, max=len(structure_ids)-1, description='Structure', continuous_update=False)
    return interact(view3d, i=s_widget)

In [ ]:
view_binding_site(structure_ids, query_chain_ids, target_chain_ids, distance=4.5);

In [ ]:
spark.stop()